## ReadCSV and preprocess

In [2]:
library(dplyr)
library(readr)
library(rjags)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: coda

Linked to JAGS 4.3.2

Loaded modules: basemod,bugs



In [3]:
# Read your data
df <- read_csv("../synthetic_data.csv")

# Create team indices (important for modeling)
teams <- sort(unique(c(df$home_team, df$away_team)))
team_index <- setNames(seq_along(teams), teams)

df <- df %>%
  mutate(
    H = team_index[home_team],
    A = team_index[away_team]
  )


Rows: 1230 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): game_date, home_team, away_team
dbl (10): game_id, fg3m_home, ftm_home, pts_home, fg3m_away, ftm_away, pts_a...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Model in JAGS for free throws

$$p = \frac{r}{r + \mu} \Rightarrow \mu = \exp(\text{linear model}) \Rightarrow r = \exp(log_r) \quad (ensures \; r > 0)$$


In [4]:
model_string_nb <- "
model {
  # Priors
  for (j in 1:(K-1)) {
    att[j] ~ dnorm(mu_att, tau_att)
    def[j] ~ dnorm(mu_def, tau_def)
  }

  mu_att ~ dnorm(0, 0.0001)
  mu_def ~ dnorm(0, 0.0001)
  tau_att ~ dgamma(0.1, 0.1)
  tau_def ~ dgamma(0.1, 0.1)

  c ~ dnorm(3, 0.0001)
  home ~ dnorm(0, 0.0001) T(0,) # strictly positive
  pH ~ dunif(0, 1)
  pA ~ dunif(0, 1)

  for (i in 1:N) {
    # Home
    HFT[i] ~ dnegbin(pH, rH[i])
    log(rH[i]) <- att[H[i]] + def[A[i]] + c + home
    
    # Away
    AFT[i] ~ dnegbin(pA, rA[i])
    log(rA[i]) <- att[A[i]] + def[H[i]] + c
  }

  # Identifiability constraints
  att[K] <- -sum(att[1:(K-1)])
  def[K] <- -sum(def[1:(K-1)]) 
}
"

In [6]:
model_string_nb <- "
model {
  # Priors
  for (j in 1:(K-1)) {
    att[j] ~ dnorm(mu_att, tau_att)
    def[j] ~ dnorm(mu_def, tau_def)
  }

  # Hyperpriors
  mu_att ~ dnorm(0, 0.0001)
  mu_def ~ dnorm(0, 0.0001)
  tau_att ~ dgamma(0.1, 0.1)
  tau_def ~ dgamma(0.1, 0.1)

  # Intercept and home advantage with more reasonable priors
  c ~ dnorm(0, 0.1)
  home ~ dnorm(0.2, 0.1) T(0,)

  # Dispersion parameter (log scale ensures r > 0)
  log_r ~ dnorm(0, 0.01)
  r <- exp(log_r)

  for (i in 1:N) {
    # Compute mean scores
    log(muH[i]) <- att[H[i]] + def[A[i]] + c + home
    log(muA[i]) <- att[A[i]] + def[H[i]] + c

    # Convert mean to JAGS p parameter
    pH[i] <- r / (r + muH[i])
    pA[i] <- r / (r + muA[i])

    # Likelihoods
    HFT[i] ~ dnegbin(pH[i], r)
    AFT[i] ~ dnegbin(pA[i], r)
  }

  # Identifiability constraints
  att[K] <- -sum(att[1:(K-1)])
  def[K] <- -sum(def[1:(K-1)])
}
"


## Prepare data for rjags

In [7]:
# Assume you've already processed your DataFrame as above
data_list <- list(
  N = nrow(df),
  K = length(teams),
  HFT = df$ftm_home,
  AFT = df$ftm_away,
  H = df$H,
  A = df$A
)

# Run model
model <- jags.model(textConnection(model_string_nb), data = data_list, n.chains = 3)
update(model, 5000)  # Burn-in

samples_nb <- coda.samples(model, variable.names = c("att", "def", "home", "c", "pH", "pA"), n.iter = 2000)
summary(samples_nb)


Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 2460
   Unobserved stochastic nodes: 65
   Total graph size: 11959

Initializing model




Iterations = 6001:6020
Thinning interval = 1 
Number of chains = 3 
Sample size per chain = 20 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

              Mean        SD  Naive SE Time-series SE
att[1]     0.15376 3.324e-01 4.292e-02      3.070e-02
att[2]     0.23488 4.749e-01 6.131e-02      8.401e-02
att[3]     0.71748 6.230e-01 8.043e-02      9.852e-02
att[4]     0.09488 4.201e-01 5.424e-02      8.809e-02
att[5]     0.25938 4.018e-01 5.188e-02      1.706e-01
att[6]     0.47998 4.791e-01 6.185e-02      1.055e-01
att[7]    -0.18896 2.855e-01 3.686e-02      7.884e-02
att[8]     0.53297 5.051e-01 6.521e-02      1.053e-01
att[9]     0.26611 2.870e-01 3.705e-02      5.209e-02
att[10]   -0.06143 1.963e-01 2.534e-02      6.203e-02
att[11]    0.48269 3.984e-01 5.144e-02      7.981e-02
att[12]   -0.34067 6.883e-01 8.886e-02      1.997e-02
att[13]    0.23183 5.647e-01 7.290e-02      5.611e-02
att[14]    0.45285 3.963e-01 5.116e-02      2.028

In [8]:
summary(samples_nb)


Iterations = 6001:6020
Thinning interval = 1 
Number of chains = 3 
Sample size per chain = 20 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

              Mean        SD  Naive SE Time-series SE
att[1]     0.15376 3.324e-01 4.292e-02      3.070e-02
att[2]     0.23488 4.749e-01 6.131e-02      8.401e-02
att[3]     0.71748 6.230e-01 8.043e-02      9.852e-02
att[4]     0.09488 4.201e-01 5.424e-02      8.809e-02
att[5]     0.25938 4.018e-01 5.188e-02      1.706e-01
att[6]     0.47998 4.791e-01 6.185e-02      1.055e-01
att[7]    -0.18896 2.855e-01 3.686e-02      7.884e-02
att[8]     0.53297 5.051e-01 6.521e-02      1.053e-01
att[9]     0.26611 2.870e-01 3.705e-02      5.209e-02
att[10]   -0.06143 1.963e-01 2.534e-02      6.203e-02
att[11]    0.48269 3.984e-01 5.144e-02      7.981e-02
att[12]   -0.34067 6.883e-01 8.886e-02      1.997e-02
att[13]    0.23183 5.647e-01 7.290e-02      5.611e-02
att[14]    0.45285 3.963e-01 5.116e-02      2.028

In [9]:
model_string_nb <- "
model {
  # Priors for attack/defense
  for (j in 1:(K-1)) {
    att[j] ~ dnorm(mu_att, tau_att)
    def[j] ~ dnorm(mu_def, tau_def)
  }

  # Identifiability constraints
  att[K] <- -sum(att[1:(K-1)])
  def[K] <- -sum(def[1:(K-1)])

  # Hyperpriors
  mu_att ~ dnorm(0, 0.0001)
  mu_def ~ dnorm(0, 0.0001)
  tau_att ~ dgamma(0.1, 0.1)
  tau_def ~ dgamma(0.1, 0.1)

  # Home advantage and baseline intercept
  c ~ dnorm(0, 0.1)
  home ~ dnorm(0.2, 0.1) T(0,)

  for (i in 1:N) {
    # Linear predictors for home and away
    log(rH[i]) <- att[H[i]] + def[A[i]] + c + home
    log(rA[i]) <- att[A[i]] + def[H[i]] + c

    # Likelihoods (with fixed p2)
    HFT[i] ~ dnegbin(p2, rH[i])
    AFT[i] ~ dnegbin(p2, rA[i])
  }
}
"


In [11]:
# Load and prepare your data
library(rjags)
df <- read_csv("../synthetic_data.csv")

# Create team index mapping
teams <- sort(unique(c(df$home_team, df$away_team)))
team_index <- setNames(seq_along(teams), teams)

df <- df %>%
  mutate(H = team_index[home_team],
         A = team_index[away_team])

# Estimate p2 from the observed data (optional)
mu_hat <- mean(c(df$fg2m_home, df$fg2m_away))
var_hat <- var(c(df$fg2m_home, df$fg2m_away))
r_est <- mu_hat^2 / (var_hat - mu_hat)  # NB moment estimator
p2 <- r_est / (r_est + mu_hat)          # Implied success probability

# Prepare data list
data_list <- list(
  N = nrow(df),
  K = length(teams),
  HFT = df$fg2m_home,     # home 2pt made
  AFT = df$fg2m_away,     # away 2pt made
  H = df$H,
  A = df$A,
  p2 = p2                 # Fixed success probability
)

# Fit the model
model <- jags.model(textConnection(model_string_nb), data = data_list, n.chains = 3)
update(model, 5000)  # Burn-in

samples_nb <- coda.samples(model, variable.names = c("att", "def", "home", "c"), n.iter = 2000)
summary(samples_nb)


Rows: 1230 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): game_date, home_team, away_team
dbl (10): game_id, fg3m_home, ftm_home, pts_home, fg3m_away, ftm_away, pts_a...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 2460
   Unobserved stochastic nodes: 64
   Total graph size: 8477

Initializing model




Iterations = 6001:8000
Thinning interval = 1 
Number of chains = 3 
Sample size per chain = 2000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

              Mean       SD  Naive SE Time-series SE
att[1]  -6.110e-01 0.042871 0.0005535      0.0015520
att[2]  -5.493e-01 0.041183 0.0005317      0.0014809
att[3]  -4.910e-01 0.040547 0.0005235      0.0014305
att[4]  -4.382e-01 0.039292 0.0005073      0.0013480
att[5]  -3.831e-01 0.039231 0.0005065      0.0013475
att[6]  -3.337e-01 0.038168 0.0004927      0.0013034
att[7]  -2.878e-01 0.036737 0.0004743      0.0011816
att[8]  -2.447e-01 0.036804 0.0004751      0.0011988
att[9]  -1.983e-01 0.034812 0.0004494      0.0010289
att[10] -1.556e-01 0.034471 0.0004450      0.0010554
att[11] -1.181e-01 0.033663 0.0004346      0.0010553
att[12] -7.915e-02 0.032545 0.0004202      0.0009630
att[13] -4.190e-02 0.032561 0.0004204      0.0009911
att[14] -6.731e-03 0.032570 0.0004205      0.0009942
att[15] 

In [ ]:
#coda::gelman.diag(samples_nb)  # > 1.1, problem with posterior

In [ ]:
par(bg = 'white')
plot(samples_nb)

ERROR: Error in eval(expr, envir, enclos): object 'samples_nb' not found


In [ ]:
#jpeg('NB_FT_trace_density.jpeg')
#par(bg = 'white')
#plot(samples_nb)
#dev.off()

## Estimate RMSE

In [ ]:
"   
    # Home
    HFT[i] ~ dpois(thetaH[i])
    log(thetaH[i]) <- att[H[i]] + def[A[i]] + c + home
    
    # Away
    AFT[i] ~ dpois(thetaA[i])
    log(thetaA[i]) <- att[A[i]] + def[H[i]] + c
"

[1] "   \n    # Home\n    HFT[i] ~ dpois(thetaH[i])\n    log(thetaH[i]) <- att[H[i]] + def[A[i]] + c + home\n    \n    # Away\n    AFT[i] ~ dpois(thetaA[i])\n    log(thetaA[i]) <- att[A[i]] + def[H[i]] + c\n"

In [ ]:
N = nrow(df)
K = length(teams)

mean_vals_att <- summary(samples_nb)$statistics[1]
mean_vals_def <- summary(samples_nb)$statistics[32]

for (i in 2:K){
    mean_vals_att <- c(mean_vals_att, summary(samples_nb)$statistics[i])
}

for (j in 33:61){
    mean_vals_def <- c(mean_vals_def, summary(samples_nb)$statistics[j])
}

c_val <- summary(samples_nb)$statistics[31]
home_val <- summary(samples_nb)$statistics[62]

In [ ]:
str(summary(samples_nb))

List of 6
 $ statistics: num [1:2522, 1:4] -0.0136 -0.0076 -0.0259 -0.0306 -0.0292 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:2522] "att[1]" "att[2]" "att[3]" "att[4]" ...
  .. ..$ : chr [1:4] "Mean" "SD" "Naive SE" "Time-series SE"
 $ quantiles : num [1:2522, 1:5] -0.197 -0.197 -0.232 -0.225 -0.239 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:2522] "att[1]" "att[2]" "att[3]" "att[4]" ...
  .. ..$ : chr [1:5] "2.5%" "25%" "50%" "75%" ...
 $ start     : num 6001
 $ end       : num 8000
 $ thin      : num 1
 $ nchain    : int 3
 - attr(*, "class")= chr "summary.mcmc"


In [ ]:
write.csv(mean_vals_att, 'NB_2p_att.csv')
write.csv(mean_vals_def, 'NB_2p_def.csv')
write.csv(c_val, 'NB_2p_c.csv')
write.csv(home_val, 'NB_2p_home.csv')

In [ ]:
# estimate theta[i] ~ exp(att[H[i]] + def[A[i]] + c + home)
theta_1 = exp(mean_vals_att[1] + mean_vals_def[4] + c_val + home_val)
theta_1

[1] 19.54429